In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!python -m pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 8.9 MB/s eta 0:00:00


In [16]:
%cd /content/gdrive/MyDrive/pytest_img/YOLO/

/content/gdrive/MyDrive/pytest_img/YOLO


In [5]:
!ls

data.yaml    nets	    README.dataset.txt	 result   sample.mp4  train  yolov5
gangnam.jpg  output_frames  README.roboflow.txt  results  test	      valid  yolov5s.pt


In [6]:
from ultralytics import YOLO
model = YOLO("../models/yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:00<00:00, 105MB/s]


In [17]:
seconds = 1 # 동영상을 읽을 시간을 초 단위로 설정
video_path = '/content/gdrive/MyDrive/pytest_img/YOLO/sample.mp4' # video path
output_dir = '/content/gdrive/MyDrive/pytest_img/YOLO/results' # 결과 저장 폴더

In [18]:
import torch

def predict(frame, iou=0.7, conf=0.25):
  results = model(
    source = frame,
    device = "0" if torch.cuda.is_available() else "cpu",
    iou = 0.7, # 바운딩 박스 필터링 임계값
    conf = 0.25, # 모델이 탐지한 객체에 대한 최소 신뢰도 기준
    verbose = False, # 추가 정보 출력 여부
  )
  result = results[0] # 첫 번째 프레임의 이미지 (현재 1개씩의 프레임만 전달됨)
  return result

In [19]:
import cv2
import numpy as np
def draw_boxes(result, frame):
  for boxes in result.boxes: # 바운딩 박스 분석 결과 순회
    x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
    label = model.names[int(classes)] # 클래스 이름을 문자형으로 받음
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 3) # 바운딩 박스 생성
    cv2.putText(frame, f'{label} {score:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
  return frame, x1, y1, x2, y2, score, classes # 처리된 이미지와 좌표, 신뢰도, 클래스 정보를 return

In [20]:
import cv2
capture = cv2.VideoCapture(video_path)

if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
  capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
fps = capture.get(cv2.CAP_PROP_FPS) # 초당 프레임 수 얻기
total_frames = int(fps * seconds) # seconds 시간 동안의 프레임 수
print('total 프레임:', total_frames) # 출력하게 되는 전체 프레임 수. 25

total 프레임: 25


In [21]:
import os
from google.colab.patches import cv2_imshow
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
frame_count = 0
#while frame_count < total_frames:
while capture.isOpened():
  ok, frame = capture.read()
  if not ok:
    print("프레임 읽기에 실패했습니다. 종료.")
    break
  result = predict(frame)
  results = draw_boxes(result, frame)

  # 렉이 걸릴수도 있기에 보여주는 코드는 주석처리
  # cv2_imshow(results[0]) # frame을 담고 있는 0번만 사용
  output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
  cv2.imwrite(output_path, results[0]) # 파일로 저장
  print("Saved to:", output_path)
  frame_count += 1
  key = cv2.waitKey(10) # 10 밀리초 동안 키 입력을 대기하고 입력된 키의 값을 반환
  if key == ord('q'): # 입력된 key 값이 'q' 라면
    print("사용자가 종료를 요청했습니다.")
    break
capture.release() # 메모리에서 자원 해제
cv2.destroyAllWindows() # 모든 창 닫기

Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0012.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/r